In [31]:
import torch
from torchvision import models
state_dict = torch.load('resnet34.pth')
model = models.resnet34(pretrained=False)
model.load_state_dict(state_dict)
model.eval()
print(model.fc.weight)

Parameter containing:
tensor([[-0.0564, -0.0554,  0.0254,  ...,  0.1835,  0.0218, -0.0415],
        [-0.0969, -0.0573,  0.0534,  ...,  0.2416, -0.0703, -0.0030],
        [-0.0257,  0.0436, -0.0174,  ...,  0.1816,  0.0426,  0.0077],
        ...,
        [-0.0432,  0.0068,  0.0601,  ..., -0.0376,  0.0455,  0.0833],
        [-0.0865, -0.0468,  0.1040,  ..., -0.0176, -0.0083,  0.0444],
        [ 0.0042, -0.0356, -0.0443,  ..., -0.0197, -0.0797,  0.1877]],
       requires_grad=True)


In [32]:
# Создание нового слоя с   1001 выходами
new_fc = torch.nn.Linear(512,  1001)

# Копирование весов из старого слоя в новый
with torch.no_grad():
    new_fc.weight[1:1001] = model.fc.weight
    new_fc.bias[1:1001] = model.fc.bias

# Замена старого слоя на новый
model.fc = new_fc


In [33]:
print(model.fc.weight.size())
print(model.fc.weight)

torch.Size([1001, 512])
Parameter containing:
tensor([[ 0.0167, -0.0090, -0.0347,  ...,  0.0146, -0.0209, -0.0345],
        [-0.0564, -0.0554,  0.0254,  ...,  0.1835,  0.0218, -0.0415],
        [-0.0969, -0.0573,  0.0534,  ...,  0.2416, -0.0703, -0.0030],
        ...,
        [-0.0432,  0.0068,  0.0601,  ..., -0.0376,  0.0455,  0.0833],
        [-0.0865, -0.0468,  0.1040,  ..., -0.0176, -0.0083,  0.0444],
        [ 0.0042, -0.0356, -0.0443,  ..., -0.0197, -0.0797,  0.1877]],
       requires_grad=True)


In [34]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# Определение стандартных трансформаций
normalize = transforms.Normalize(mean=[0.485,  0.456,  0.406], std=[0.229,  0.224,  0.225])
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Путь к директории с изображениями
data_dir = './classes'
dataset = ImageFolder(data_dir, transform=transform)


In [35]:
from torch.utils.data import DataLoader

# Параметры DataLoader
batch_size =  64
shuffle = True

# Создание DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [36]:
num_features = model.fc.in_features
num_epochs = 3
# Заморозка нижних слоев модели
for param in model.parameters():
    param.requires_grad = False

# Размораживание верхних слоев для обучения
for param in model.fc.parameters():
    param.requires_grad = True

# Определение функции потерь и оптимизатора
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Обучение модели*
for epoch in range(num_epochs):
    running_loss =  0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)  # Умножаем на размер батча
    epoch_loss = running_loss / len(dataset)  # Делим на общее количество элементов в наборе данных
    print('Epoch {}/{}: Loss: {:.4f}'.format(epoch, num_epochs -  1, epoch_loss))


Epoch 0/2: Loss: 11.5722
Epoch 1/2: Loss: 6.9905
Epoch 2/2: Loss: 0.8528


In [39]:
# Сохранение обученной модели
torch.save(model.state_dict(), 'trained_resnet34.pth')
